In [ ]:
import cv2 as cv
import numpy as np
from skimage.feature import hog
import pandas as pd
import numpy as np
#"/Users/Mehr/Documents/projects/tree/1.jpg"
import cv2 as cv


In [ ]:
from skimage import io, transform, img_as_ubyte
import os
import numpy as np

def resize_image_skimage(input_dir, output_dir, target_size):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(input_dir, filename)
            try:
                image = io.imread(image_path)
                resized_image = transform.resize(image, target_size, anti_aliasing=True)
                
                # Convert RGBA to RGB
                if resized_image.shape[-1] == 4:
                    resized_image = resized_image[..., :3]
                
                resized_image = img_as_ubyte(resized_image)
                output_path = os.path.join(output_dir, os.path.splitext(filename)[0] + ".jpg")
                io.imsave(output_path, resized_image)
            except Exception as e:
                print(f"Error processing {image_path}: {e}")

input_directory = '/Users/Mehr/Documents/projects/tree/'
output_directory = '/Users/Mehr/Documents/projects/tree/resized/'
target_size = (224, 224)  # Example target size

#resize_image_skimage(input_directory, output_directory, target_size)


In [ ]:
import cv2 as cv
from skimage.feature import hog
import os

def extract(path):
    image_path = os.path.join("/Users/Mehr/Documents/projects/tree/resized/", path)
    image = cv.imread(image_path)

    # Check if the image was loaded successfully
    if image is None:
        print(f"Error: Could not load image at {image_path}")
        return None

    # Convert to grayscale
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    features, hog_image = hog(image, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=True, feature_vector=True)
    return features

# Example usage
path = "1.jpg"  # Replace with your actual image filename
features = extract(path)
if features is not None:
    print("Features extracted successfully")
else:
    print("Failed to extract features")


In [ ]:
import cv2
import numpy as np
import pandas as pd
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score

# Define the feature extraction function
def extract_features(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features, hog_image = hog(image, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=True, feature_vector=True)
    return features

# Example data
# data = pd.DataFrame({
#     'filenames': ['image1.jpg', 'image2.jpg'],  # Replace with your actual filenames
#     'tree_count': [5, 3]  # Replace with your actual tree counts
# })

data=pd.read_csv("/Users/Mehr/Documents/projects/tree/resized/tree.csv")
data=data[:4]

def extract(path):
    image=cv.imread("/Users/Mehr/Documents/projects/tree/resized/"+path)
    image=cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    features, hog_image = hog(image, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=True,feature_vector=True)
    return features

# Extract features and check lengths
data['features'] = data['filenames'].apply(extract)
feature_lengths = data['features'].apply(len)

# Ensure all feature vectors have the same length
if len(feature_lengths.unique()) == 1:
    X = np.array(data['features'].tolist())
    y = data['tree_count'].values
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train the model
    model = RandomForestRegressor(n_estimators=100)
    model.fit(X_train, y_train)
    
    # Make predictions
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print(f"Mean Squared Error: {mse}")
else:
    print("Inconsistent feature lengths detected:", feature_lengths.unique())


In [ ]:
def predict_tree_count(image_path,model):
    features=extract(image_path)
    if features is None:
            print(f"Error: Could not extract features from {image_path}.")
            return None
    features=np.array(features).reshape(1,-1)
    prediction=model.predict(features)
    return prediction[0]

new_image_path = "/Users/Mehr/Documents/projects/tree/test1/test.jpg"
    
    # Predict the tree count in the new image
predicted_count = predict_tree_count(new_image_path, model)
print(f"Predicted number of trees in the new image: {predicted_count}")


In [ ]:
import cv2
import numpy as np
import pandas as pd
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
import os
import xgboost as xgb


# Define the feature extraction function
def extract_features(image_path):
    if not os.path.exists(image_path):
        print(f"Error: Image path {image_path} does not exist.")
        return None
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Failed to read image {image_path}.")
        return None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features, hog_image = hog(image, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=True, feature_vector=True)
    return features

# Load the data
data = pd.read_csv("/Users/Mehr/Documents/projects/tree/resized/tree.csv")
#data = data[:14]

# Extract features and check lengths
data['features'] = data['filenames'].apply(lambda x: extract_features("/Users/Mehr/Documents/projects/tree/resized/" + x))
data = data.dropna(subset=['features'])  # Drop rows where features extraction failed
feature_lengths = data['features'].apply(len)

# Ensure all feature vectors have the same length
if len(feature_lengths.unique()) == 1:
    X = np.array(data['features'].tolist())
    y = data['tree_count'].values
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train the model
    #model = RandomForestRegressor(n_estimators=200,max_depth=20,min_samples_leaf=3,max_features='sqrt')
    #model=xgb.XGBRegressor(n_estimators=100,max_depth=20,learning_rate=0.01)
    #model=GradientBoostingRegressor(n_estimators=200,max_depth=20,min_samples_leaf=3,max_features='sqrt')
    model=SVR(kernel='rbf',C=100,epsilon=0.1)
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print(f"Mean Squared Error: {mse}")
    
    # Define a function to predict the number of trees in a new image
    def predict_tree_count(image_path, model):
        features = extract_features(image_path)
        if features is None:
            print(f"Error: Could not extract features from {image_path}.")
            return None
        features = np.array(features).reshape(1, -1)  # Reshape to 2D array
        prediction = model.predict(features)
        return prediction[0]
    
    # Path to the new image
    new_image_path = "/Users/Mehr/Documents/projects/tree/resized/test.jpg"
    
    # Predict the tree count in the new image
    predicted_count = predict_tree_count(new_image_path, model)
    if predicted_count is not None:
        print(f"Predicted number of trees in the new image: {predicted_count}")
else:
    print("Inconsistent feature lengths detected:", feature_lengths.unique())


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import cv2
import os

# Define the feature extraction function
def extract_features(image_path):
    if not os.path.exists(image_path):
        print(f"Error: Image path {image_path} does not exist.")
        return None
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Failed to read image {image_path}.")
        return None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features, hog_image = hog(image, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=True, feature_vector=True)
    return features

# Load the data
data = pd.read_csv("/Users/Mehr/Documents/projects/tree/resized/tree.csv")
data = data[:4]

# Extract features and check lengths
data['features'] = data['filenames'].apply(lambda x: extract_features("/Users/Mehr/Documents/projects/tree/resized/" + x))
data = data.dropna(subset=['features'])  # Drop rows where features extraction failed
feature_lengths = data['features'].apply(len)


def best_model_params(models,X,y):    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    
    mse=[]
    predict_tree=[]
    best_params=[]
    model_name=[]
    estimator=[]
    for model in models:
    # Define the parameter grid
        if isinstance(model,RandomForestRegressor):
            param_grid = {
            'n_estimators': [30,50,100],
            'max_depth': [2,3,5],
            'min_samples_split': [2, 5,10],
        }
        elif isinstance(model,xgb.XGBRegressor):
            param_grid = {
            'n_estimators': [30, 50,100],
            'max_depth': [2,3,5],
            'min_samples_split': [2, 5,10],
            'learning_rate': [.0001,.001,0.01],
        }
        elif isinstance(model,GradientBoostingRegressor):
            param_grid = {
            'n_estimators': [30, 50,100],
            'max_depth': [2,3,5],
            'min_samples_split': [2, 5,10],
            'learning_rate': [.0001,.001,0.01],
        }
        elif isinstance(model,SVR):
            param_grid = {
            'C': [0.1, 1, 10, 100],
            'epsilon': [0.01, 0.1, 1],
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'gamma': ['scale', 'auto']
        }
        
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=1, verbose=1)  # Disable parallel processing
        grid_search.fit(X_train, y_train)
        
        model_name.append(type(model).__name__)
        estimator.append(grid_search.best_estimator_)
        best_model = grid_search.best_estimator_
        
        predictions = best_model.predict(X_test)
        
        mse.append(mean_squared_error(y_test, predictions))
        predict_tree.append(predictions)
        best_params.append(best_model.get_params())
        
    return pd.DataFrame({'model_name':model_name,'estimator':estimator,'mse':mse,'predict_tree':predict_tree,'best_params':best_params})
    

# Ensure all feature vectors have the same length
if len(feature_lengths.unique()) == 1:
    X = np.array(data['features'].tolist())
    y = data['tree_count'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    #model = RandomForestRegressor()
    #model=xgb.XGBRegressor()
    #model=GradientBoostingRegressor()
    #model=SVR()
    models=[RandomForestRegressor(),xgb.XGBRegressor(),GradientBoostingRegressor(),SVR()]
    best_models=best_model_params(models,X,y)
    #best_model=xgb.XGBRegressor(min_samples_split=1, learning_rate=0.01,max_depth=3,n_estimators=50)
    
    print(best_models)
    
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # #best_model.fit(X_train, y_train) 
    
    # predictions = best_model.predict(X_test)
    # mse = mean_squared_error(y_test, predictions)
    # print(f"Mean Squared Error: {mse}")
    
    # Define a function to predict the number of trees in a new image
    def predict_tree_count(image_path, models):
        features = extract_features(image_path)
        if features is None:
            print(f"Error: Could not extract features from {image_path}.")
            return None
        features = np.array(features).reshape(1, -1)  # Reshape to 2D array
        predictions=[]
        model_names=[]
        for model in models['estimator']:
            predictions.append(model.predict(features))
            model_names.append(type(model).__name__)   
        return pd.DataFrame({'model_name':model_names,'predictions':predictions})
    
    # Path to the new image
    new_image_path = "/Users/Mehr/Documents/projects/tree/resized/test.jpg"
    
    # Predict the tree count in the new image
    predicted_count = predict_tree_count(new_image_path, best_models)
    if predicted_count is not None:
        #print(f"Predicted number of trees in the new image: {predicted_count}")
        best_models['predictions']=predicted_count['predictions']
        best_models
else:
    print("Inconsistent feature lengths detected:", feature_lengths.unique())


In [ ]:
best_models